# Bayesian Nested Mixture Model

This notebook illustrate how to build and train a Bayesian Nested Mixture Model with the [beer framework](https://github.com/beer-asr/beer). A Nested Mixture Model is simply a "mixture of mixture", the rationale behind this model is that the clusters of a mixture model may have arbitrary complex distribution approximated by yet another mixture model.

In [1]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.append('../')

import copy

import beer
import numpy as np

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import LinearAxis, Range1d
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

## Data

We generate a synthetic data:


In [2]:
# First cluster.
mean = np.array([-1.5, 4]) 
cov = np.array([[.75, 0], [0, 2.]])
data1 = np.random.multivariate_normal(mean, cov, size=100)

# Second cluster.
mean = np.array([5, 5]) 
cov = np.array([[2, 1], [1, .75]])
data2 = np.random.multivariate_normal(mean, cov, size=100)

# Third cluster.
mean = np.array([-1.5, -4]) 
cov = np.array([[.75, 0], [0, 2.]])
data3 = np.random.multivariate_normal(mean, cov, size=100)

# Fourth cluster.
mean = np.array([5, -5]) 
cov = np.array([[2, 1], [1, .75]])
data4 = np.random.multivariate_normal(mean, cov, size=100)

# Merge everything to get the finale data set.
data = np.vstack([data1, data2, data4, data3])
np.random.shuffle(data)

In [50]:
# Mean, variance of the data to scale the figure.
mean = data.mean(axis=0)
var = data.var(axis=0)
std_dev = np.sqrt(max(var))
x_range = (mean[0] - 2 * std_dev, mean[0] + 2 * std_dev)
y_range = (mean[1] - 2 * std_dev, mean[1] + 2 * std_dev)

fig = figure(title='Data', width=400, height=400,
             x_range=x_range, y_range=y_range)
fig.circle(data[:, 0], data[:, 1])

show(fig)

## Model Creation

We create two types of mixture model: one whose (Normal) components have full covariance matrix and the other whose (Normal) components have diagonal covariance matrix.

In [85]:
# We use the global mean/cov. matrix of the data to initialize the mixture.
p_mean = data.mean(axis=0)
p_cov = np.cov(data.T)

# Create the models.
args = {'dim':2, 'mean':p_mean, 'cov': p_cov, 'prior_count':1, 'random_init':True}
gmm_diag = beer.NestedMixture.create(5, 3, beer.NormalDiagonalCovariance.create, args, prior_count=1e-3)
gmm_full = beer.NestedMixture.create(3, 5, beer.NormalFullCovariance.create, args, prior_count=1)

## Variational Bayes Training 

In [86]:
# Callback to monitor the training progress.
elbos, llhs, klds = [], [], []
def callback(elbo, llh, kld):
    elbos.append(elbo)
    llhs.append(llh)
    klds.append(kld)

# Train the GMM with diagonal cov. matrix components.
gmm_diag.fit(data, max_epochs=200, callback=callback)
elbos_diag = copy.deepcopy(elbos[1:])
llhs_diag =  copy.deepcopy(llhs[1:])
klds_diag = copy.deepcopy(klds[1:])

# Re-initialize the training progress.
elbos, llhs, klds = [], [], []

# Train the GMM with full cov. matrix components.
gmm_full.fit(data, max_epochs=200, callback=callback)
elbos_full = copy.deepcopy(elbos[1:])
llhs_full =  copy.deepcopy(llhs[1:])
klds_full = copy.deepcopy(klds[1:])

# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
fig.line(np.arange(len(elbos_diag)), elbos_diag, legend='GMM (diag)', color='blue')
fig.line(np.arange(len(elbos_full)), elbos_full, legend='GMM (full)', color='red')
fig.legend.location = 'bottom_right'

show(fig)

In [47]:
print(gmm_full.weights)
np.exp(gmm_full.c_posterior_weights[0].grad_lognorm()).sum(), np.exp(gmm_full.c_posterior_weights[1].grad_lognorm()).sum()

[ 0.00139861  0.2502736   0.74832779]


(0.62301372218724649, 0.98321159575976425)

In [ ]:
for i in range(3):
    print(gmm_diag.c_posterior_weights[i].natural_params.sum())

In [ ]:
132.556520815 + 129.473122932 + 137.970356253

In [ ]:
gmm_full.weights, np.exp(gmm_diag.c_posterior_weights[0].grad_lognorm()).sum()

In [ ]:
fig1 = figure(title='GMM (diag)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, gmm_diag, color='blue')

fig2 = figure(title='GMM (full)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig2, gmm_full, color='red')

grid = gridplot([[fig1, fig2]])
show(grid)

## Split-VB algorithm

The VB algorithm is sensitive to the initialization of the posterior distribution and may end up in a local maximum. The basic strategy used above is to initialize randomly the posterior distribution of the Normal components. This may not be desirable in all situtation as it can lead to poor covering of the space by the clusters. An alternative is the "split-VB" algorithm. This algorithm consists in fitting a Normal distribution to the data and then to split this distribtution into 2 other Normal distribution. Then, the process is repeated: we train using VB these 2 Normal distributions and split them again, ... until we reach the desire number of components. The splitting is done by moving the mean of the Normal distribution along the axis with the highest variability.

Here is an example of splitting a mixture model:

In [ ]:
gmm_diag_split = gmm_diag.split()
gmm_full_split = gmm_full.split()

fig1 = figure(title='GMM (diag)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, gmm_diag, color='blue')

fig2 = figure(title='GMM (full)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig2, gmm_full, color='red')

fig3 = figure(title='GMM splitted (diag)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig3.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig3, gmm_diag_split, color='blue')

fig4 = figure(title='GMM splitted (full)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig4.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig4, gmm_full_split, color='red')

grid = gridplot([[fig1, fig2], [fig3, fig4]])
show(grid)

Let's compare the VB algorithm with random initialization to the split-VB training.

In [ ]:
# First we run 10 times the standard VB training.
elbos_vb = []
for i in range(1):
    args = {'dim':2, 'mean':p_mean, 'cov': p_cov, 'prior_count':1, 'random_init':True}
    gmm_full = beer.Mixture.create(16, beer.NormalFullCovariance.create, args, prior_count=1e-3)
    
    elbos = []
    def callback(elbo, llh, kld):
        elbos.append(elbo)
    gmm_full.fit(data, max_epochs=200, callback=callback)
    elbos_vb.append(elbos[-1])
    
# New we run 10 times the split VB training.
elbos_split_vb = []
for i in range(1):
    final_elbo = 0
    def callback(elbo, llh, kld):
        global final_elbo
        final_elbo = elbo
        
    args = {'dim':2, 'mean':p_mean, 'cov': p_cov, 'prior_count':1, 'random_init':False}
    gmm_full_split = beer.Mixture.create(1, beer.NormalFullCovariance.create, args, prior_count=1e-6)
    gmm_full_split.fit(data, max_epochs=100)

    for k in range(2):
        gmm_full_split = gmm_full_split.split()
        gmm_full_split.fit(data, max_epochs=100, callback=callback)
    gmm_full_split.fit(data, max_epochs=1000, callback=callback)
    elbos_split_vb.append(final_elbo)
        
        

In [ ]:
print(elbos_vb)
print(elbos_split_vb)

In [ ]:
fig1 = figure(title='GMM (full)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig1, gmm_full, color='blue')

fig2 = figure(title='GMM splitted (full)', x_range=x_range, y_range=y_range,
              width=400, height=400)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plotting.plot_gmm(fig2, gmm_full_split, color='red')

grid = gridplot([[fig1, fig2]])
show(grid)